<a href="https://colab.research.google.com/github/Jack1Hayes/BA_-review_Analysis/blob/main/analysis_of_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyldavis


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import spacy
import re
import seaborn as sns
import matplotlib.pyplot as plt
import glob

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
Review_table = pd.read_csv('/content/Clean_data.csv')
Review_table = Review_table[Review_table.Recommended != 'yes']

In [ ]:
Review_table.head()

,verification,reviews,header,country,Name,date,Aircraft,Type Of Traveller,Seat Type,Route,...,Recommended,neg score,neu score,pos score,textblob,textblob_pol,Noun Phrases,Verbs,docs,token
0,✅ Trip Verified,Vancouver to Delhi via London. We were booke...,"""felt misguided and misinformed""",Canada,D Bartul,28th December 2022,NaN,Family Leisure,Economy Class,Vancouver to Delhi via London,...,no,15.2,81.3,3.5,0.402693,0.020545,"[' Vancouver', 'Delhi', 'London', 'We', 'Vanc...","['book', 'fly', 'receive', 'inform', 'call', '...",Vancouver to Delhi via London. We were booke...,"[' ', 'vancouver', 'delhi', 'london', 'book',..."
2,✅ Trip Verified,Another BA Shambles. Started off well with e...,"""treat your customers like idiots""",United Kingdom,Amanda Edgar,22nd December 2022,Boeing 777-300,Couple Leisure,First Class,London to Male,...,no,13.4,71.5,15.1,0.537500,0.293125,"[' Another BA Shambles', 'excellent check', '...","['start', 'avoid', 'call', 'jostle', 'get', 'c...",Another BA Shambles. Started off well with e...,"[' ', 'ba', 'shambles', 'start', 'excellent',..."
3,Not Verified,BA cancelled my flight home to Heathrow on D...,"""cancelled only 4 hours before take off""",United Kingdom,Andrew Wilkes,20th December 2022,A320,Solo Leisure,Economy Class,Keflavik to London Heathrow,...,no,13.2,84.0,2.8,0.436667,-0.227500,"['BA', 'my flight home', 'Heathrow', 'Dec 19th...","['cancel', 'cancel', 'take', 'struggle', 'get'...",BA cancelled my flight home to Heathrow on D...,"[' ', 'ba', 'cancel', 'flight', 'home', 'heat..."
4,Not Verified,"BA cancelled my flight home, the last flight...","""have to use other airlines now""",United Kingdom,Nick Walker,19th December 2022,NaN,Business,Business Class,London Heathrow to Frankfurt,...,no,10.3,84.9,4.8,0.364394,-0.020455,"['BA', 'my flight home', 'the day', 'Heathrow'...","['cancel', 'try', 'push', 'want', 'try', 'cros...","BA cancelled my flight home, the last flight...","[' ', 'ba', 'cancel', 'flight', 'home', 'flig..."
5,✅ Trip Verified,"Turned up 3.5 hours in advance, Terminal 5 at...","""my suitcase didn’t arrive with me""",United Kingdom,E King,19th December 2022,NaN,Solo Leisure,Economy Class,London Heathrow to Bangkok via Doha,...,no,13.5,80.2,6.3,0.374123,-0.021053,"['advance', 'Terminal', 'London Heathrow', 'pe...","['turn', 'try', 'check', 'man', 'stress', 'pus...","Turned up 3.5 hours in advance, Terminal 5 at...","[' ', 'turn', '3.5', 'hour', 'advance', 'termi..."


In [ ]:
import string
def clean_round_1(text):
  text = text.lower()
  text=re.sub("i'm", 'i am',text)
  text=re.sub("he's", "he is",text)
  text=re.sub("she's", "she is",text)
  text=re.sub("what's", "what is",text)
  text=re.sub("where's", "where is",text)
  text=re.sub("don't", "do not",text)
  text=re.sub("\'ll", " will",text)
  text=re.sub("\ve", " have",text)
  text=re.sub("\re", " are",text)
  text=re.sub("\d", "would",text)
  text=re.sub("wont", "will not",text)
  text=re.sub("can't", "can not",text)
  text= text.translate(str.maketrans( '', '', string.punctuation))
  return text

In [ ]:
Review_table['clean_1'] = Review_table['reviews'].apply(clean_round_1)

In [ ]:
def clean_round_2(text):
  text = " ".join([word for word in text.split()])
  doc = nlp(text)
  text = [word.lemma_ for word in doc]
  text = [word for word in text if word not in nlp.Defaults.stop_words]
  text = ",".join([word for word in text])
  return text

In [ ]:
Review_table['clean_2'] = Review_table['clean_1'].apply(clean_round_2)

In [ ]:
def remove_words(text):
  final_words = []
  for word in text:
    if len(word) > 2:
      final_words.append(word)
  return final_words

In [ ]:
Review_table['clean_3'] = Review_table['clean_2'].str.strip('()').str.split(',')
type(Review_table['clean_3'].iloc[0])

list

In [ ]:
text_data = Review_table['clean_3']
Review_table['clean_3'].head()

0    [vancouver, delhi, london, book, fly, vancouve...
2    [ba, shamble, start, excellent, check, twould,...
3    [ba, cancel, flight, home, heathrow, dec, woul...
4    [ba, cancel, flight, home, flight, day, heathr...
5    [turn, wouldwould, hour, advance, terminal, lo...
Name: clean_3, dtype: object

In [ ]:
id2word = corpora.Dictionary(text_data)
texts = text_data
print(texts)

0      [vancouver, delhi, london, book, fly, vancouve...
2      [ba, shamble, start, excellent, check, twould,...
3      [ba, cancel, flight, home, heathrow, dec, woul...
4      [ba, cancel, flight, home, flight, day, heathr...
5      [turn, wouldwould, hour, advance, terminal, lo...
                             ...                        
991    [st, lucia, gatwick, night, flight, decide, fl...
992    [fort, lauderdale, gatwick, efficient, courtes...
994    [london, las, vegas, disappoint, ba, flight, d...
998    [family, I, travel, cape, town, london, wouldw...
999    [gatwick, madeira, flight, leave, time, arrive...
Name: clean_3, Length: 635, dtype: object


In [ ]:
corpus = [id2word.doc2bow(word) for word in texts]


In [ ]:
print(corpus[:1][0][:30])

[(0, 2), (1, 1), (2, 1), (3, 2), (4, 2), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 1), (11, 5), (12, 1), (13, 1), (14, 1), (15, 3), (16, 1), (17, 9), (18, 4), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1)]


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=7,
                                            random_state=100,
                                            update_every=1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha = 'auto')

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds = 'mmds', R=20)
pyLDAvis.save_html(vis, 'output_filename.html')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.188917 -0.194624       1        1  40.288118
1     -0.033246 -0.316811       2        1  23.556232
6     -0.283392  0.071692       3        1  15.871909
0     -0.140201  0.229161       4        1  12.592537
3      0.252614 -0.124587       5        1   3.383154
5      0.250198  0.106617       6        1   2.852002
2      0.142945  0.228553       7        1   1.456047, topic_info=          Term         Freq        Total Category  logprob  loglift
1            I  2820.000000  2820.000000  Default  20.0000  20.0000
440      class   223.000000   223.000000  Default  19.0000  19.0000
439   business   225.000000   225.000000  Default  18.0000  18.0000
206       seat   653.000000   653.000000  Default  17.0000  17.0000
43      flight  1257.000000  1257.000000  Default  16.0000  16.0000
...        ...          ...          ...      ...      ...      ...
1724   ryanair     6.148760     9.838088   Topic7  -4.6896   3.7594
1470   layover     4.221142     6.414299   Topic7  -5.0658   3.8110
3200       abu     6.679199    12.853081   Topic7  -4.6069   3.5749
3202     dhabi     6.679199    12.853081   Topic7  -4.6069   3.5749
275   terminal     5.386117    61.352170   Topic7  -4.8221   1.7966

[286 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.970435     "
0         5  0.016448     "
967       2  0.960567     '
1         1  0.532206     I
1         2  0.442146     I
...     ...       ...   ...
280       2  0.185235  year
280       4  0.412569  year
1203      2  0.257644  york
1203      6  0.708520  york
405       3  0.968330     –

[439 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 7, 1, 4, 6, 3])

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=text_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('nCoherence Score: ', coherence_lda)


nCoherence Score:  0.48290578918834876


In [ ]:
vis.topic_info[vis.topic_info.Term == 'service']

,Term,Freq,Total,Category,logprob,loglift
105,service,188.401150,437.603033,Topic1,-4.5876,0.0664
105,service,104.047242,437.603033,Topic2,-4.6447,0.0093
105,service,68.961503,437.603033,Topic3,-4.6612,-0.0071
105,service,75.930540,437.603033,Topic4,-4.3334,0.3206
